In [ ]:
pip install ultralytics

In [ ]:
from ultralytics import YOLO
import numpy as np

In [ ]:
## connecting to the google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def calculate_midpoint(box_coordinates):
    x_min, y_min, x_max, y_max = box_coordinates
    x_mid = (x_min + x_max) / 2
    y_mid = (y_min + y_max) / 2
    return x_mid, y_mid

## Tracking Ball and handling oclusion

In [ ]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def draw_trajectory(video_path, yolo_model):
    # Initialize the video capture object
    output_video_path = "/content/drive/MyDrive/CV/cricket_segmentation/cricket_segmentation/vid2/annotated_video3.mp4"

    video_capture = cv2.VideoCapture(video_path)

    # Get the width and height of the video
    frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Initialize the video writer object
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (frame_width, frame_height))

    # Initialize the list of midpoints
    midpoints = []

    # Create a list of classes
    class_names = ['ball']

    # Loop over the frames in the video
    while True:
        # Read the frame from the video capture object
        success, frame = video_capture.read()

        # If the frame is not successfully read, break from the loop
        if not success:
            break

        # Find the midpoint of the detected ball
        result = yolo_model.predict(frame, classes=32, save = True, max_det=1,conf=0.15)
        coordinates = result[0].boxes.xyxy.cpu().tolist()
        midpoint = None
        if len(coordinates) != 0:
          midpoint = calculate_midpoint(coordinates[0])

        # If the ball is detected for the first time, add the midpoint to the list of midpoints
        if midpoint is not None:
            if len(midpoints) == 0:
                print("Mid point added for the first time")
                midpoints.append(midpoint)
        # If the ball has been detected in previous frames, add the midpoint to the list of midpoints and connect it to the previous midpoint
            else:
              if len(midpoints) > 0:
                  midpoints.append(midpoint)
                  # for i in range(1, len(midpoints)):
                  #   previous_midpoint = midpoints[i - 1]
                  #   cv2.line(frame, (int(previous_midpoint[0]),int(previous_midpoint[1])),(int(midpoints[i][0]),int(midpoints[i][1])), (0, 255, 0), thickness=8)

        if len(midpoints) > 0:
          for i in range(1, len(midpoints)):
            previous_midpoint = midpoints[i - 1]
            cv2.line(frame, (int(previous_midpoint[0]),int(previous_midpoint[1])),(int(midpoints[i][0]),int(midpoints[i][1])), (0, 255, 0), thickness=8)

        # Write the frame to the video writer object
        out.write(frame)

        # Display the frame
        cv2_imshow(frame)

        # If the user presses `q`, break from the loop
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # Release the video capture object and video writer object
    video_capture.release()
    out.release()

if __name__ == '__main__':
    # Draw the trajectory of the cricket ball in the video
    model = YOLO('yolov8x.pt')

    draw_trajectory('/content/drive/MyDrive/CV/cricket_segmentation/cricket_segmentation/vid2/test8.mp4', model)
